In [5]:
import os
import django
import pandas as pd
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:

from django.db import transaction
from analysis.models import PlayerPerformance
from analysis.models import MatchStats

class PlayersData:
    def __init__(self):
        pass
        
    def Data(self):
        
        fixture_data = MatchStats.objects.all().values()
        df = pd.DataFrame(list(fixture_data))
        stats_df = pd.json_normalize(df['stats'])
        mycols = ["lineups.match_id","lineups.response_code",
                  "match_info.data.event.homeTeam.id","match_info.data.event.awayTeam.id",
                  "match_info.data.event.homeTeam.name","match_info.data.event.awayTeam.name",
                  "lineups.data.home.players","lineups.data.away.players",
                  "lineups.data.home.missingPlayers","lineups.data.away.missingPlayers"]

        data = stats_df[mycols]
        data.columns = ["match_id","code_response",
                        "home_id","away_id",
                        "home_name","away_name",
                        "home_players","away_players",
                        "home_missing","away_missing"]
        
        home_players_expanded = data[['match_id', 'home_players','home_name','home_id']].explode('home_players').reset_index(drop=True)
        home_players_normalized = pd.json_normalize(home_players_expanded['home_players'])
        
        home_players_normalized['match_id'] = home_players_expanded['match_id']
        home_players_normalized['teamname'] = home_players_expanded['home_name']
        home_players_normalized['teamid'] = home_players_expanded['home_id']
        
        home_players_normalized['team'] = 'home'
        home_players_normalized.dropna(subset=['player.position'], inplace=True)
        away_players_expanded = data[['match_id', 'away_players','away_name','away_id']].explode('away_players').reset_index(drop=True)
        
        away_players_normalized = pd.json_normalize(away_players_expanded['away_players'])
        
        away_players_normalized['match_id'] = away_players_expanded['match_id']
        away_players_normalized['teamname'] = away_players_expanded['away_name']
        away_players_normalized['teamid'] = away_players_expanded['away_id']
        
        away_players_normalized['team'] = 'away'
        away_players_normalized.dropna(subset=['player.position'], inplace=True)

        players_normalized = pd.concat([home_players_normalized,away_players_normalized])

    
        original_columns = [
                            "shirtNumber", "jerseyNumber", "position", "substitute", "player.name", "player.slug", 
                            "player.shortName", "player.position", "player.jerseyNumber", "player.height", 
                            "player.userCount", "player.id", "player.country.alpha2", "player.country.alpha3", 
                            "player.country.name", "player.country.slug", "player.marketValueCurrency", 
                            "player.dateOfBirthTimestamp", "statistics.totalPass", "statistics.accuratePass", 
                            "statistics.totalLongBalls", "statistics.accurateLongBalls", "statistics.goalAssist", 
                            "statistics.duelWon", "statistics.wasFouled", "statistics.goodHighClaim", 
                            "statistics.savedShotsFromInsideTheBox", "statistics.saves", "statistics.minutesPlayed", 
                            "statistics.touches", "statistics.rating", "statistics.possessionLostCtrl", 
                            "statistics.ratingVersions.original", "statistics.ratingVersions.alternative", 
                            "statistics.totalCross", "statistics.accurateCross", "statistics.aerialWon", 
                            "statistics.duelLost", "statistics.challengeLost", "statistics.totalContest", 
                            "statistics.blockedScoringAttempt", "statistics.totalTackle", "player.firstName", 
                            "player.lastName", "statistics.aerialLost", "statistics.wonContest", 
                            "statistics.bigChanceCreated", "statistics.totalClearance", "statistics.interceptionWon", 
                            "statistics.fouls", "statistics.keyPass", "player.fieldTranslations.nameTranslation.ar", 
                            "player.fieldTranslations.shortNameTranslation.ar", "statistics.outfielderBlock", 
                            "statistics.dispossessed", "statistics.bigChanceMissed", "statistics.onTargetScoringAttempt", 
                            "captain", "statistics.shotOffTarget", "statistics.hitWoodwork", "statistics.totalOffside", 
                            "statistics.punches", "statistics.totalKeeperSweeper", "statistics.accurateKeeperSweeper", 
                            "statistics.goals", "statistics.errorLeadToAShot", "statistics.penaltyConceded", 
                            "statistics.lastManTackle", "statistics.ownGoals", "statistics.expectedGoals", 
                            "statistics.expectedAssists", "statistics.penaltyWon", "statistics.penaltyMiss", 
                            "statistics.clearanceOffLine", "statistics.goalsPrevented", "statistics.errorLeadToAGoal", 
                            "statistics.penaltySave", "statistics.penaltyShootoutSave", "statistics.penaltyShootoutGoal", 
                            "statistics.penaltyShootoutMiss", "match_id", "teamname", "teamid", "team"
                        ]

        
        for col in original_columns:
               if col not in players_normalized.columns:
                    players_normalized[col] = 0
            
        players = players_normalized[original_columns]


        new_columns = [
                        "shirt_number", "jersey_number", "position", "substitute", "player_name", "player_slug", 
                        "player_short_name", "player_position", "player_jersey_number", "player_height", 
                        "player_user_count", "player_id", "country_alpha2", "country_alpha3", 
                        "country_name", "country_slug", "market_value_currency", 
                        "date_of_birth_timestamp", "PAS_total_passes", "PAS_accurate_passes", 
                        "PAS_long_balls", "PAS_accurate_long_balls", "ATT_goal_assist", 
                        "MUC_duel_won", "MUC_fouls_suffered", "KAL_good_high_claim", 
                        "KAL_saved_shots_from_inside_the_box", "KAL_saves", "minutes_played", 
                        "touches", "rating", "MUC_possession_lost", 
                        "rating_original", "rating_alternative", 
                        "PAS_crosses", "PAS_accurate_crosses", "MUC_aerial_duel_won", 
                        "MUC_duel_lost", "MUC_challenge_lost", "MUC_total_contest", 
                        "DEF_blocked_scoring_attempt", "DEF_tackles", "player_first_name", 
                        "player_last_name", "MUC_aerial_lost", "MUC_won_contest", 
                        "ATT_big_chance_created", "DEF_clearances", "DEF_interceptions", 
                        "MUC_fouls_committed", "PAS_key_passes", "name_translation_ar", 
                        "short_name_translation_ar", "DEF_outfielder_block", 
                        "MUC_dispossessed", "ATT_big_chance_missed", "ATT_on_target_scoring_attempt", 
                        "captain", "ATT_shot_off_target", "ATT_hit_woodwork", "MUC_offsides", 
                        "KAL_punches", "KAL_total_keeper_sweeper", "KAL_accurate_keeper_sweeper", 
                        "ATT_goals", "MUC_error_lead_to_a_shot", "DEF_penalty_conceded", 
                        "DEF_last_man_tackle", "DEF_own_goals", "ATT_expected_goals", 
                        "ATT_expected_assists", "ATT_penalty_won", "ATT_penalty_miss", 
                        "DEF_clearance_off_line", "KAL_goals_prevented", "MUC_error_lead_to_a_goal", 
                        "KAL_penalty_save", "KAL_penalty_shootout_save", "KAL_penalty_shootout_goal", 
                        "KAL_penalty_shootout_miss", "match_id", "team_name", "team_id", "team"
                    ]


        players.columns = new_columns
        playersdata = players.fillna(0)
        playersdata.replace('', 0, inplace=True)

        
         
        return playersdata

    def add_to_database(self):

        players = self.Data()

    # Veri eklemek veya güncellemek için döngüyle her satırı işliyoruz
        for _, row in players.iterrows():
            defaults = {
                
                'shirt_number': row.get('shirt_number', None) if pd.notna(row.get('shirt_number')) else None,
                'jersey_number': row.get('jersey_number', None) if pd.notna(row.get('jersey_number')) else None,
                'position': row.get('position', None) if pd.notna(row.get('position')) else None,
                'substitute': row.get('substitute', None) if pd.notna(row.get('substitute')) else None,
                'player_name': row.get('player_name', None) if pd.notna(row.get('player_name')) else None,
                'player_slug': row.get('player_slug', None) if pd.notna(row.get('player_slug')) else None,
                'player_short_name': row.get('player_short_name', None) if pd.notna(row.get('player_short_name')) else None,
                'player_position': row.get('player_position', None) if pd.notna(row.get('player_position')) else None,
                'player_jersey_number': row.get('player_jersey_number', None) if pd.notna(row.get('player_jersey_number')) else None,
                'player_height': row.get('player_height', None) if pd.notna(row.get('player_height')) else None,
                'player_user_count': row.get('player_user_count', None) if pd.notna(row.get('player_user_count')) else None,
                'country_alpha2': row.get('country_alpha2', None) if pd.notna(row.get('country_alpha2')) else None,
                'country_alpha3': row.get('country_alpha3', None) if pd.notna(row.get('country_alpha3')) else None,
                'country_name': row.get('country_name', None) if pd.notna(row.get('country_name')) else None,
                'country_slug': row.get('country_slug', None) if pd.notna(row.get('country_slug')) else None,
                'market_value_currency': row.get('market_value_currency', None) if pd.notna(row.get('market_value_currency')) else None,
                'date_of_birth_timestamp': row.get('date_of_birth_timestamp', None) if pd.notna(row.get('date_of_birth_timestamp')) else None,
            
                # ATT (Attack) Stats
                'ATT_goals': row.get('ATT_goals', None) if pd.notna(row.get('ATT_goals')) else None,
                'ATT_goal_assist': row.get('ATT_goal_assist', None) if pd.notna(row.get('ATT_goal_assist')) else None,
                'ATT_shot_off_target': row.get('ATT_shot_off_target', None) if pd.notna(row.get('ATT_shot_off_target')) else None,
                'ATT_hit_woodwork': row.get('ATT_hit_woodwork', None) if pd.notna(row.get('ATT_hit_woodwork')) else None,
                'ATT_big_chance_created': row.get('ATT_big_chance_created', None) if pd.notna(row.get('ATT_big_chance_created')) else None,
                'ATT_big_chance_missed': row.get('ATT_big_chance_missed', None) if pd.notna(row.get('ATT_big_chance_missed')) else None,
                'ATT_on_target_scoring_attempt': row.get('ATT_on_target_scoring_attempt', None) if pd.notna(row.get('ATT_on_target_scoring_attempt')) else None,
                'ATT_expected_goals': row.get('ATT_expected_goals', None) if pd.notna(row.get('ATT_expected_goals')) else None,
                'ATT_expected_assists': row.get('ATT_expected_assists', None) if pd.notna(row.get('ATT_expected_assists')) else None,
                'ATT_penalty_won': row.get('ATT_penalty_won', None) if pd.notna(row.get('ATT_penalty_won')) else None,
                'ATT_penalty_miss': row.get('ATT_penalty_miss', None) if pd.notna(row.get('ATT_penalty_miss')) else None,
            
                # PAS (Pass) Stats
                'PAS_total_passes': row.get('PAS_total_passes', None) if pd.notna(row.get('PAS_total_passes')) else None,
                'PAS_accurate_passes': row.get('PAS_accurate_passes', None) if pd.notna(row.get('PAS_accurate_passes')) else None,
                'PAS_key_passes': row.get('PAS_key_passes', None) if pd.notna(row.get('PAS_key_passes')) else None,
                'PAS_crosses': row.get('PAS_crosses', None) if pd.notna(row.get('PAS_crosses')) else None,
                'PAS_accurate_crosses': row.get('PAS_accurate_crosses', None) if pd.notna(row.get('PAS_accurate_crosses')) else None,
                'PAS_long_balls': row.get('PAS_long_balls', None) if pd.notna(row.get('PAS_long_balls')) else None,
                'PAS_accurate_long_balls': row.get('PAS_accurate_long_balls', None) if pd.notna(row.get('PAS_accurate_long_balls')) else None,
            
                # MUC (Duels and Fouls) Stats
                'MUC_duel_won': row.get('MUC_duel_won', None) if pd.notna(row.get('MUC_duel_won')) else None,
                'MUC_aerial_duel_won': row.get('MUC_aerial_duel_won', None) if pd.notna(row.get('MUC_aerial_duel_won')) else None,
                'MUC_duel_lost': row.get('MUC_duel_lost', None) if pd.notna(row.get('MUC_duel_lost')) else None,
                'MUC_aerial_lost': row.get('MUC_aerial_lost', None) if pd.notna(row.get('MUC_aerial_lost')) else None,
                'MUC_won_contest': row.get('MUC_won_contest', None) if pd.notna(row.get('MUC_won_contest')) else None,
                'MUC_challenge_lost': row.get('MUC_challenge_lost', None) if pd.notna(row.get('MUC_challenge_lost')) else None,
                'MUC_total_contest': row.get('MUC_total_contest', None) if pd.notna(row.get('MUC_total_contest')) else None,
                'MUC_possession_lost': row.get('MUC_possession_lost', None) if pd.notna(row.get('MUC_possession_lost')) else None,
                'MUC_fouls_committed': row.get('MUC_fouls_committed', None) if pd.notna(row.get('MUC_fouls_committed')) else None,
                'MUC_fouls_suffered': row.get('MUC_fouls_suffered', None) if pd.notna(row.get('MUC_fouls_suffered')) else None,
                'MUC_offsides': row.get('MUC_offsides', None) if pd.notna(row.get('MUC_offsides')) else None,
                'MUC_dispossessed': row.get('MUC_dispossessed', None) if pd.notna(row.get('MUC_dispossessed')) else None,
                'MUC_error_lead_to_a_shot': row.get('MUC_error_lead_to_a_shot', None) if pd.notna(row.get('MUC_error_lead_to_a_shot')) else None,
                'MUC_error_lead_to_a_goal': row.get('MUC_error_lead_to_a_goal', None) if pd.notna(row.get('MUC_error_lead_to_a_goal')) else None,

    # DEF (Defensive) Stats
                'DEF_clearances': row.get('DEF_clearances', None) if pd.notna(row.get('DEF_clearances')) else None,
                'DEF_blocked_scoring_attempt': row.get('DEF_blocked_scoring_attempt', None) if pd.notna(row.get('DEF_blocked_scoring_attempt')) else None,
                'DEF_tackles': row.get('DEF_tackles', None) if pd.notna(row.get('DEF_tackles')) else None,
                'DEF_interceptions': row.get('DEF_interceptions', None) if pd.notna(row.get('DEF_interceptions')) else None,
                'DEF_last_man_tackle': row.get('DEF_last_man_tackle', None) if pd.notna(row.get('DEF_last_man_tackle')) else None,
                'DEF_penalty_conceded': row.get('DEF_penalty_conceded', None) if pd.notna(row.get('DEF_penalty_conceded')) else None,
                'DEF_own_goals': row.get('DEF_own_goals', None) if pd.notna(row.get('DEF_own_goals')) else None,
                'DEF_clearance_off_line': row.get('DEF_clearance_off_line', None) if pd.notna(row.get('DEF_clearance_off_line')) else None,
                          'DEF_outfielder_block': row.get('DEF_outfielder_block', None) if pd.notna(row.get('DEF_outfielder_block')) else None,
            
                # KAL (Kaleci) Stats
                'KAL_good_high_claim': row.get('KAL_good_high_claim', None) if pd.notna(row.get('KAL_good_high_claim')) else None,
                'KAL_saved_shots_from_inside_the_box': row.get('KAL_saved_shots_from_inside_the_box', None) if pd.notna(row.get('KAL_saved_shots_from_inside_the_box')) else None,
                'KAL_saves': row.get('KAL_saves', None) if pd.notna(row.get('KAL_saves')) else None,
                'KAL_punches': row.get('KAL_punches', None) if pd.notna(row.get('KAL_punches')) else None,
                'KAL_total_keeper_sweeper': row.get('KAL_total_keeper_sweeper', None) if pd.notna(row.get('KAL_total_keeper_sweeper')) else None,
                'KAL_accurate_keeper_sweeper': row.get('KAL_accurate_keeper_sweeper', None) if pd.notna(row.get('KAL_accurate_keeper_sweeper')) else None,
                'KAL_goals_prevented': row.get('KAL_goals_prevented', None) if pd.notna(row.get('KAL_goals_prevented')) else None,
                'KAL_penalty_save': row.get('KAL_penalty_save', None) if pd.notna(row.get('KAL_penalty_save')) else None,
                'KAL_penalty_shootout_save': row.get('KAL_penalty_shootout_save', None) if pd.notna(row.get('KAL_penalty_shootout_save')) else None,
                'KAL_penalty_shootout_goal': row.get('KAL_penalty_shootout_goal', None) if pd.notna(row.get('KAL_penalty_shootout_goal')) else None,
                'KAL_penalty_shootout_miss': row.get('KAL_penalty_shootout_miss', None) if pd.notna(row.get('KAL_penalty_shootout_miss')) else None,

                # Match and Team Information
                'team_name': row.get('team_name', None) if pd.notna(row.get('team_name')) else None,
                'team_id': row.get('team_id', None) if pd.notna(row.get('team_id')) else None,
                'team': row.get('team', None) if pd.notna(row.get('team')) else None,
            
                # Other Statistics
                'minutes_played': row.get('minutes_played', None) if pd.notna(row.get('minutes_played')) else None,
                'touches': row.get('touches', None) if pd.notna(row.get('touches')) else None,
                'rating': row.get('rating', None) if pd.notna(row.get('rating')) else None,
                'rating_original': row.get('rating_original', None) if pd.notna(row.get('rating_original')) else None,
                'rating_alternative': row.get('rating_alternative', None) if pd.notna(row.get('rating_alternative')) else None,
            
                # Additional Player Information
                'player_first_name': row.get('player_first_name', None) if pd.notna(row.get('player_first_name')) else None,
                'player_last_name': row.get('player_last_name', None) if pd.notna(row.get('player_last_name')) else None,

                            }

            PlayerPerformance.objects.update_or_create(
                    match_id=row.get('match_id'),
                    player_id=row.get('player_id'),
                    defaults=defaults
                          )
        print("Player Performance Data Guncellendi.")

        


d = PlayersData()
data = d.add_to_database()
data

Player Performance Data Guncellendi.


In [ ]:
player_data = PlayerPerformance.objects.all().values()
dfp = pd.DataFrame(list(player_data))
dfp